In [1]:
import numpy as np

from sparsepoly import SparseFactorizationMachineRegressor

import warnings

warnings.filterwarnings("ignore")

import os
from math import ceil
import json

# Create True Model Parameter

In [2]:
np.set_printoptions(precision=5)
n_features = 100
n = 200
random_state = np.random.RandomState(0)
W_true = np.zeros((n_features, n_features))
for i in range(0, 8 * n_features // 10, 10):
    W_true[i : i + 10, i : i + 10] = 0.5
W_true[range(n_features), range(n_features)] = 0
print(np.sum(W_true))
print(W_true)

360.0
[[0.  0.5 0.5 ... 0.  0.  0. ]
 [0.5 0.  0.5 ... 0.  0.  0. ]
 [0.5 0.5 0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


# Create data distribution and dataset

In [3]:
mean = np.zeros(n_features)
Cov = np.zeros((n_features, n_features))
for i in range(0, 8 * n_features // 10, 10):
    Cov[i : i + 10, i : i + 10] = 0.2

Cov[range(n_features), range(n_features)] = 1

In [4]:
X = random_state.multivariate_normal(mean, Cov, size=n)
y = np.sum(np.dot(X, W_true) * X, axis=1) + random_state.normal(0, 0.1, n)

# Evaluation functions

In [5]:
def estimation_error(W_true, P, scaling=True):
    n_features = W_true.shape[0]
    W_estimated = np.dot(P, P.T)
    W_estimated[range(n_features), range(n_features)] = 0.0
    diff = (2.0 * W_true - W_estimated) ** 2
    l2_error = np.sqrt(np.sum(diff) / 2.0)
    if scaling:
        l2_error /= np.sqrt(np.sum((2.0 * W_true) ** 2) / 2.0)
    return l2_error


def support_recovery(W_true, P):
    n_features = W_true.shape[0]
    W_estimated = np.dot(P, P.T)
    W_estimated[range(n_features), range(n_features)] = 0.0

    supp_true = W_true != 0
    supp_estimated = W_estimated != 0

    tp = (np.sum(supp_true * supp_estimated)) // 2
    tn = (np.sum(np.invert(supp_true) * np.invert(supp_estimated)) - n_features) // 2
    fp = (np.sum(np.invert(supp_true) * supp_estimated)) // 2
    fn = (np.sum(supp_true * np.invert(supp_estimated))) // 2
    if (tp + fp) == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        fscore = 0
    else:
        fscore = 2 * precision * recall / (precision + recall)
    return {
        "fscore": fscore,
        "pssr": (fp + fn) == 0,
        "nnz": np.count_nonzero(W_estimated) // 2,
    }

# Comparison with Tuned hyperparameters

In [6]:
n_trial = 10
max_iter = 400
reg_solvers = {"squaredl12": "pcd", "l1": "pcd", "squaredl21": "pbcd", "l21": "pbcd"}
for regularizer, solver in reg_solvers.items():
    print(regularizer)
    for beta_power in range(-2, 2):
        beta = 10.0**beta_power
        for gamma_power in range(-2, 2):
            gamma = 10.0**gamma_power
            score = {"error": 0, "fscore": 0, "pssr": 0, "nnz": 0.0}
            for n in range(n_trial):
                fm = SparseFactorizationMachineRegressor(
                    n_components=30,
                    fit_linear=False,
                    beta=2 * beta,
                    gamma=gamma,
                    regularizer=regularizer,
                    solver=solver,
                    mean=True,
                    max_iter=max_iter,
                    tol=1e-3,
                    random_state=n,
                    verbose=0,
                )
                fm.fit(X, y)
                score["error"] += estimation_error(W_true, fm.P_[0].T, True)
                results = support_recovery(W_true, fm.P_[0].T)

                for key, val in results.items():
                    score[key] += val

            for key in score.keys():
                score[key] /= n_trial
            print(f"  beta: {beta}, gamma: {gamma} \n    {score}")
    print()

squaredl12
  beta: 0.01, gamma: 0.01 
    {'error': 0.21794254857983453, 'fscore': 0.4142246540293689, 'pssr': 0.0, 'nnz': 1394.9}
  beta: 0.01, gamma: 0.1 
    {'error': 0.1292800283697564, 'fscore': 0.9835703737199344, 'pssr': 0.2, 'nnz': 372.1}
  beta: 0.01, gamma: 1.0 
    {'error': 0.5211666522346629, 'fscore': 0.9814324876255481, 'pssr': 0.0, 'nnz': 346.9}
  beta: 0.01, gamma: 10.0 
    {'error': 1.0, 'fscore': 0.0, 'pssr': 0.0, 'nnz': 0.0}
  beta: 0.1, gamma: 0.01 
    {'error': 0.02512966619635897, 'fscore': 0.557660109578737, 'pssr': 0.0, 'nnz': 934.9}
  beta: 0.1, gamma: 0.1 
    {'error': 0.07182262347255, 'fscore': 0.9863013698630139, 'pssr': 0.0, 'nnz': 370.0}
  beta: 0.1, gamma: 1.0 
    {'error': 0.4998716218263247, 'fscore': 0.9877432450388384, 'pssr': 0.0, 'nnz': 351.3}
  beta: 0.1, gamma: 10.0 
    {'error': 1.0, 'fscore': 0.0, 'pssr': 0.0, 'nnz': 0.0}
  beta: 1.0, gamma: 0.01 
    {'error': 0.42891742696512675, 'fscore': 0.13991792274934295, 'pssr': 0.0, 'nnz': 4785.